# Demonstration of the implementation concept of CDT

In [ ]:
import numpy as np

In [ ]:
n_I, n_J, n_a = 5, 3, 2
L_x, L_y = 4, 2
# L_x, L_y = 10, 6
# n_I, n_J, n_a = 2, 2, 2
# L_x, L_y = 1, 1
X_aIJ = np.mgrid[0:L_x:complex(n_I),0:L_y:complex(n_J)]
X_IJa = np.einsum('aIJ->IJa', X_aIJ)
X_IJa.shape

Given a column index $I = [0, n_I-1\}$, row index $J = [0, n_J-1]$ and direction index $a = [0,1]$ 
the index expression
$$
\mathcal{G}_{IJa} = (1-a)I + a J
$$
introduces a grid index map rendering the horizontal/column indexes of individual grid nodes for $a=0$ and vertical / row indexes for $a=1$.

$$
 a = 0 \rightarrow (\zeta = 1, \eta = 0)
$$
$$
 a = 1 \rightarrow (\zeta = 0, \eta = 1)
$$
$$
\zeta = 1-a
$$
$$
\eta = a
$$

In [ ]:
I, J, a = np.arange(n_I), np.arange(n_J), np.arange(n_a)
G_aIJ = (np.einsum('a, I->aI', (1-a), I)[:,:, None] + 
         np.einsum('a, J->aJ', a, J)[:,None, :])
G_aIJ

Given the step length 
$$
\Delta X_a = \left[\frac{L_x}{n_I}, \frac{L_y}{n_J} \right],
$$
the distance between grid points is
The coordinates of all nodes are expressed as
$$
X_{aIJ} = \Delta X_{a} \, \mathcal{G}_{aIJ}
$$

In [ ]:
delta_X_a = np.array([L_x/(n_I-1), L_y/(n_J-1)])
X_aIJ = np.einsum('aIJ, a->aIJ', G_aIJ, delta_X_a)
X_aIJ

The enumeration of nodes within a single element can be defined for the $\xi$ and $\eta$ directions using the index 
map
$$
\mathcal{g}_{ai} = \left[
\begin{array}{cccc}
0,& 1, &1, & 0 \\ 0,& 0,& 1,& 1
\end{array}
\right]
$$

In [ ]:
g_ai = np.array([[0, 1, 1, 0], [0, 0, 1, 1]], dtype=np.int_)

By introducing the element indexes $E = [0, n_I-2]$ and $F = [0, n_J-2]$ in the 
horizontal and vertical direction, we can introduce the index map identifying
the local element nodes enumerated counter clock-wise in each element of the grid as
$$
\mathcal{H}_{aEFi} = \mathcal{G}_{aEF} + g_{ai}
$$

In [ ]:
G_aEF = G_aIJ[:,:-1,:-1]
G_aEF

In [ ]:
H_aEFi = G_aEF[:, :, :, None] + g_ai[:, None, None, :]
H_EFia = np.einsum('aEFi->EFia', H_aEFi)
X_IJa = np.einsum('aIJ->IJa', X_aIJ)
H_EFia[0,0], X_IJa.shape, len(I)

$$
  X_{EFia} = X_{I=\mathcal{H}_{0EFi}, I=\mathcal{H}_{1EFi}}
$$

In [ ]:
X_EFia = X_IJa[tuple(H_aEFi)]

In [ ]:
X_aEFi = np.einsum('EFia->aEFi', X_EFia)

## Nodal coordinates and quadrature points of an element

In [ ]:
delta_rs = np.eye(2, dtype=np.int_)

In [ ]:
xi_ir = np.array([[-1, -1], [1, -1], [1, 1], [-1, 1]])

$$
 \eta_{mr} = \frac{1}{\sqrt{3}} \xi{mr}
$$

In [ ]:
eta_mr = 3**(-1/2) * xi_ir
eta_mr

## Bilinear Lagrange shape functions

### Dimensional directions explicitly referenced in the product expression
$$
N_{mi} = \frac{1}{4}(1 + \eta_{m0} \xi_{i0})(1 + \eta_{m1} \xi_{i1})
$$

In [ ]:
N1_im = (
    (1 + np.einsum('m,i->mi', eta_mr[:,0], xi_ir[:,0]))* 
    (1 + np.einsum('m,i->mi', eta_mr[:,1], xi_ir[:,1]))
) / 4

### Dimensional directions included in the index operator
$$
N_i(\eta_r) 
=
\frac{1}{4}(
1 + \eta_0 \xi_{i0} + \eta_1 \xi_{i1} + \eta_0 \xi_{i0} \eta_1 \xi_{i1}
)
$$

$$
N_{im}
=
\frac{1}{4}\left(
1 + \eta_{mr} \xi_{ir} + \frac{1}{2}(1 - \delta_{rs}) \eta_{ms} \xi_{is} \eta_{mr} \xi_{ir}
\right)
$$

In [ ]:
N_im = (1 + 
  np.einsum('mr,ir->im', eta_mr, xi_ir) +
  np.einsum('rs,ms,is,mr,ir->im', (1-delta_rs), eta_mr, xi_ir, eta_mr, xi_ir) / 2
)/4
N_im - N1_im

## Derivatives of the shape functions w.r.t. parametric coordinates

$$
\frac{\partial N_i }{\partial \eta_0}
= 
\frac{1}{4}( \xi_{i0} + \eta_1 \xi_{i1} ), \;\;
\frac{\partial N_i }{\partial \eta_1}
= 
\frac{1}{4}( \xi_{i1} + \eta_0 \xi_{i0} )
$$

$$
N_{i,s}(\eta_r)
= 
\frac{1}{4}( \xi_{is} +
(1-\delta_{rs}) \xi_{is} \eta_{r} \xi_{ir}
)
$$

$$
N_{im,s}
= 
\frac{1}{4}
\left[ \xi_{is} +
(1-\delta_{rs}) \, \xi_{is} \eta_{rm} \xi_{ir}
\right]
$$

In [ ]:
dN_ims = (
    xi_ir[:,None,:] + 
    np.einsum('rs,is, mr,ir->ims', (1 - delta_rs), xi_ir, eta_mr, xi_ir)
) / 4

## Verification of shape functions

## Jacobi matrix

In [ ]:
J_EFmas = np.einsum(
 'ims,EFia->EFmas',
 dN_ims, X_EFia
)

In [ ]:
inv_J_EFmsa = np.linalg.inv(J_EFmas)

In [ ]:
delta_ab = np.eye(2)
Diff1_abcd = 0.5 * (
    np.einsum('ac,bd->abcd', delta_ab, delta_ab) +
    np.einsum('ad,bc->abcd', delta_ab, delta_ab)
)

In [ ]:
B_EFimabc = np.einsum(
    'abcd,ims,EFmsd->EFimabc',
    Diff1_abcd, dN_ims, inv_J_EFmsa
)

In [ ]:
B_EFimabc.shape

In [ ]:
X_EFia[0,0]

In [ ]:
eps_EFmab = np.einsum(
    'EFimabc,EFic->EFmab',
    B_EFimabc, X_EFia * 0.0001
)
eps_EFmab[0,0]

In [ ]:
n_mp = 30
alpha = np.linspace(0, 2 * np.pi, n_mp)
MPN_en = np.array([np.cos(alpha), np.sin(alpha)])

In [ ]:
MPW = 1/n_mp

In [ ]:
MPNN_nij = np.einsum('en,fn->nef', MPN_en, MPN_en)

In [ ]:
MPTT_nijr = 0.5 * (
        np.einsum('en,fr -> nefr', MPN_en, np.eye(2)) +
        np.einsum('fn,re -> nfer', MPN_en, np.eye(2)) - 2 *
        np.einsum('en,fn,gn -> nefg', MPN_en, MPN_en, MPN_en)
)

In [ ]:
e_N_EFmn = np.einsum('nij,...ij->...n', MPNN_nij, eps_EFmab)
e_T_EFmna = np.einsum('nija,...ij->...na', MPTT_nijr, eps_EFmab)

In [ ]:
e_N_pos_EFmn = (np.abs(e_N_EFmn) + e_N_EFmn) / 2
e_TT_EFmn = np.einsum('...ni,...ni -> ...n', e_T_EFmna, e_T_EFmna)

In [ ]:
c_T = 0
e_equiv_EFmn = np.sqrt(e_N_pos_EFmn * e_N_pos_EFmn + c_T * e_TT_EFmn)
epsilon_0 = 5e-5
epsilon_f = 0.001

In [ ]:
I = np.where(e_equiv_EFmn >= epsilon_0)
kappa_EFmn = np.zeros_like(e_equiv_EFmn)
omega_EFmn = np.zeros_like(e_equiv_EFmn)
kappa_EFmn[I] = e_equiv_EFmn[I]

In [ ]:
omega_EFmn[I] = np.array(
    1.0 - (epsilon_0 / kappa_EFmn[I] * np.exp(-1.0 * (kappa_EFmn[I] - epsilon_0) / (epsilon_f - epsilon_0))))

In [ ]:
phi_EFmn = np.sqrt(1.0 - omega_EFmn)

In [ ]:
phi_EFmab = np.einsum('...n,nab->...ab', phi_EFmn, MPNN_nij) / n_mp

In [ ]:
delta = np.eye(2)
beta_EFmijkl = 0.25 * (np.einsum('...ik,jl->...ijkl', phi_EFmab, delta) +
                      np.einsum('...il,jk->...ijkl', phi_EFmab, delta) +
                      np.einsum('...jk,il->...ijkl', phi_EFmab, delta) +
                      np.einsum('...jl,ik->...ijkl', phi_EFmab, delta))

In [ ]:
E_ = 24000
nu_ = 0.18

In [ ]:
la = E_ * nu_ / ((1. + nu_) * (1. - 2. * nu_))
mu = E_ / (2. + 2. * nu_)
delta = np.eye(2)
D_abef = (np.einsum(',ij,kl->ijkl', la, delta, delta) +
          np.einsum(',ik,jl->ijkl', mu, delta, delta) +
          np.einsum(',il,jk->ijkl', mu, delta, delta))

In [ ]:
D_EFmijab = np.einsum(
    '...ijab, abef, ...cdef -> ...ijcd',
    beta_EFmijkl, D_abef, beta_EFmijkl
)

sig_EFmab = np.einsum('...abef,...ef -> ...ab', D_EFmijab, eps_EFmab)

In [ ]:
sig_EFmab[0,0]